In [2]:
import glob
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
files=glob.glob('../Data/Tested_Data/*')

# Reading a multiple json files from a single json file 'ProductSample.json'.
product=[]
with open(files[0]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        product.append(i)
        
# Firstly cleaning the data by converting files into proper json format files by some replacements and 
# then Making a list of Tuples containg data of brand Rubie's Costume Co.
productDataframe=[]
brand_List=['Rubie&#39;s Costume Co']
for x in product:
    try:
        y=x.replace("'",'"')
        jdata=json.loads(y)
        if jdata['brand'] in brand_List:
            productDataframe.append((jdata['asin'],jdata['title'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.  
Product_dataset=pd.DataFrame(productDataframe,columns=['Asin','Title'])

In [8]:
prod_List=Product_dataset.Asin.unique().tolist()

In [9]:
# Reading a multiple json files from a single json file 'ReviewSample.json'.
review=[]
with open(files[1]) as data_file:
    data=data_file.read()
    for i in data.split('\n'):
        review.append(i)
        
# Making a list of Tuples containg all the data of json files.
reviewDataframe=[]
for x in review:
    try:
        jdata=json.loads(x)
        reviewDataframe.append((jdata['reviewerID'],jdata['asin'],jdata['overall'])) 
    except:
        pass
    
# Creating a dataframe using the list of Tuples got in the previous step.     
review_dataset=pd.DataFrame(reviewDataframe,columns=['Reviewer_ID','Asin','Rating'])    

In [10]:
Working_dataset=review_dataset[review_dataset.Asin.isin(prod_List)]

Working_dataset=pd.merge(Working_dataset,Product_dataset,on='Asin',how='inner')

Working_dataset=Working_dataset[['Reviewer_ID','Asin','Title','Rating']]
Model = Working_dataset.pivot_table(index='Reviewer_ID',columns='Title',values='Rating')

In [14]:
def pearson(x1,x2):
    x1_cor=x1-x1.mean()
    x2_cor=x2-x2.mean()
    return np.sum(x1_cor * x2_cor)/np.sqrt(np.sum(x1_cor**2) * np.sum(x2_cor**2))

def get_recommendations(product_id,M,num):
    recomend=[]
    for asin in M.columns:
        if asin==product_id:
            continue
        cor=pearson(M[product_id],M[asin])
        if np.isnan(cor):
            continue
        else:
            recomend.append((asin,cor))
    recomend.sort(key=lambda tup: tup[1],reverse=True)
    return recomend[:num]

def escape(t):
    """HTML-escape the text in `t`."""
    return (t
        .replace("&amp;","&").replace("&lt;","<").replace("&gt;",">")
        .replace("&#39;","'").replace("&quot;",'"')
        )

In [19]:
rec=get_recommendations('300 Movie Spartan Shield',Model,5)

In [20]:
Recommendation=[]
for x in rec:
    if x[1] > 0:
        Recommendation.append((escape(x[0]),x[1]))

result=pd.DataFrame(Recommendation,columns=['Product Title','Correlation '])    

In [22]:
result.to_csv('../Analysis/Analysis_5/Recommendation.csv')

In [23]:
result

,Product Title,Correlation
0,300 Movie Spartan Deluxe Vinyl Helmet,0.223277
1,Toynk Toys - 300- Spartan Sword,0.069275
